PLR with a lightswitch
======================

This example shows how we can measure the PLR with a lightswitch in a dark room and instantly plot


In [7]:
import sys
sys.path.insert(0, '../')
import os
import os.path as op
from time import sleep

import numpy as np

from pyplr import stlab
from pyplr.pupil import PupilCore, unpack_data_pandas
from pyplr.pipeline import butterworth_series
from pyplr.plr import plr_parameters, plot_plr

baseline = 2.
duration = 5.

# set up pupil
p = PupilCore()

# annotation to be sent when you flip the switch
annotation = p.new_annotation('LIGHT_ON')
    
# start LightStamper and PupilGrabber
lst_future = p.light_stamper(annotation, threshold=15, timeout=6.)
pgr_future = p.pupil_grabber(topic='pupil.1.3d', seconds=10)
    
# baseline
sleep(baseline)

# flip that switch!
# wait a second...
# now flip it off!
# now wait for the pgr_future
    
# retrieve and process pupil data
data = unpack_data_pandas(pgr_future.result())
data = butterworth_series(
    data, filt_order=3, cutoff_freq=4/(120/2), fields=['diameter_3d'])
    
# lightstamper timestamp
ts = lst_future.result()[1]
    
# find the closest timestamp in the pupil data
idx = (np.abs(ts - data.index)).argmin()

# trim the data
data = (data.iloc[idx-240:idx+600]
            .reset_index(inplace=True))

# parametrise and plot
plr_parameters(data.diameter_3d, 120, idx, 0.01)
plot_plr(data.diameter_3d, 120, idx, 1, vel_acc=True)

Waiting for a light to stamp...Grabbing 10 seconds of pupil.1.3d

Light stamped on frame.world at 791532.0192130001
PupilGrabber done grabbing 10 seconds of pupil.1.3d


AttributeError: 'NoneType' object has no attribute 'diameter_3d'

In [5]:
data = unpack_data_pandas(pgr_future.result())


In [6]:
data

,id,topic,method,norm_pos,diameter,confidence,sphere,projected_sphere,circle_3d,diameter_3d,ellipse,location,model_confidence,theta,phi
timestamp,,,,,,,,,,,,,,,
791472.839241,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.46022697824184067, 0.47818106636707736]",21.270731,1.0,"{'center': [6.006848907361025, 2.0998050068029...","{'center': [126.40558209357206, 106.6337657913...","{'center': [-0.795724229060216, 1.134388632173...",2.892747,"{'center': [88.36357982243341, 100.18923525752...","[88.36357982243341, 100.18923525752115]",1.0,1.663828,-2.288187
791472.847182,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4597558465724658, 0.47774277908257856]",21.077288,1.0,"{'center': [6.006848907361025, 2.0998050068029...","{'center': [126.40558209357206, 106.6337657913...","{'center': [-0.7975130505634294, 1.17497721283...",2.866976,"{'center': [88.27312254191344, 100.27338641614...","[88.27312254191344, 100.27338641614492]",1.0,1.659906,-2.288104
791472.855052,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4601827115106117, 0.47784169172487945]",21.048648,1.0,"{'center': [6.006848907361025, 2.0998050068029...","{'center': [126.40558209357206, 106.6337657913...","{'center': [-0.7920236637897018, 1.16857274798...",2.862493,"{'center': [88.35508061003745, 100.25439518882...","[88.35508061003745, 100.25439518882314]",1.0,1.660525,-2.287449
791472.863328,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4599723409285325, 0.4781614200553733]",20.957316,1.0,"{'center': [6.006848907361025, 2.0998050068029...","{'center': [126.40558209357206, 106.6337657913...","{'center': [-0.7885745606933643, 1.15710972162...",2.850467,"{'center': [88.31468945827824, 100.19300734936...","[88.31468945827824, 100.19300734936832]",1.0,1.661632,-2.287094
791472.871055,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4602644880622999, 0.4783418603535293]",21.205181,1.0,"{'center': [6.006848907361025, 2.0998050068029...","{'center': [126.40558209357206, 106.6337657913...","{'center': [-0.7520250827449821, 1.12987819723...",2.883828,"{'center': [88.37078170796158, 100.15836281212...","[88.37078170796158, 100.15836281212236]",1.0,1.664264,-2.282631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791482.803199,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4416862696900563, 0.4640905883828028]",22.233604,1.0,"{'center': [5.847011225879715, 1.9822619855788...","{'center': [125.29674260964576, 105.9353765635...","{'center': [-0.8069683023142993, 0.34183458158...",3.077932,"{'center': [84.80376378049081, 102.89460703050...","[84.80376378049081, 102.89460703050186]",1.0,1.729310,-2.276288
791482.811173,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4418078093031464, 0.46498821687563074]",22.069693,1.0,"{'center': [5.847011225879715, 1.9822619855788...","{'center': [125.29674260964576, 105.9353765635...","{'center': [-0.7861879836323595, 0.30846691268...",3.055231,"{'center': [84.82709938620411, 102.72226235987...","[84.82709938620411, 102.7222623598789]",1.0,1.732562,-2.274077
791482.819144,1,pupil.1.3d,pye3d 0.0.4 real-time,"[0.4426407433232445, 0.465719640588765]",21.921521,1.0,"{'center': [5.847011225879715, 1.9822619855788...","{'center': [125.29674260964576, 105.9353765635...","{'center': [-0.7808716784981424, 0.31817317327...",3.033502,"{'center': [84.98702271806295, 102.58182900695...","[84.98702271806295, 102.58182900695711]",1.0,1.731616,-2.273267
